# 循环神经网络

注意，本笔记是邱锡鹏老师[《神经网络与深度学习》](https://nndl.github.io)第六章1、2、3、4的一个总结。

循环神经网络是一类具有短期记忆能力的神经网络。在循环神经网络中，神经元不但可以接受其他神经元的信息，也可以接受自身的信息，形成具有环路的网络结构。和前馈神经网络相比，循环神经网络更加符合生物神经网络的结构。

## RNN的神经元

给定一个输入时序数据$x_{1:T} = (x_1, x_2, \ldots, x_t, \ldots, x_T)$， RNN通过下列公式更新隐藏层的活性值$h_t$
$$
h_t = f(h_{t - 1}, x_t)\tag{1}
$$
其中$h_0 = 0$,$f(\cdot)$为一个非线性函数，可以使一个前馈网络。

在让我用通俗一些的语言描述一遍，一个RNN的神经元就是一个可以接受当前的时序数据$x_t$和神经元上一时刻的活性值$h_{t - 1}$作为输入的非线性函数。

从数学上说，公式(1)可以看作一个动力系统，因此，隐藏层的活性值$h_t$也可以被称为状态，理论上，循环神经网络可以近似任意的非线性动力系统。

## 简单循环神经网络

令向量$x_t\in R^M$表示在时刻$t$时网络的输入, $h_t\in R^D$表示隐藏层状态.简单循环网络在时刻$t$的更新公式为
$$
z_t = Uh_{t - 1} + Wx_t + b\tag{2}
$$

$$
h_t = f(z_t)\tag{3}
$$

其中$z_t$为隐藏层的净输入。$f(\cdot)$是非线性激活函数，通常为Sigmoid或者Tanh函数，公式(2)、(3也经常直接写为

$$
h_t = f(Uh_{t - 1} + Wx_t + b)\tag{4}
$$

值得一说的是，简单循环神神经网络的**一个隐藏层只有一个神经元**。

### RNN的应用场景
总结下来，RNN的应用场景可以分为三种情况
1. 序列到类别
  序列到类别是将序列数据应用到分类问题。输入为序列，输出为类别，比如在文本分类中，输入数据为token序列，输出为文本的类别。我们假设分类器为$g(\cdot)$,那么使用如下公式进行分类
  $$\hat{y} = g(h_T)\tag{5}$$
  当然，也可以对整个序列的所有状态进行平均，并用这个平均状态作为整个序列的表示
  $$\hat{y} = g(\frac 1T\displaystyle\sum_{t = 1}^Th_t)\tag{6}$$

2. 同步的序列到序列
  同步的序列到序列主要适用于序列标注，每一个输入都必须对应一个输出，输入和输出的长度相同。同样假设分类器为$g(\cdot)$，那么，使用如下公式对每个时刻进行标注
  $$\hat{y}_t = g(h_t)\tag{7}$$

3. 异步的序列到序列
  异步的序列到序列也被称为Encoder-Decoder模型，输入和输出序列不需要有严格的对应关系。比如机器翻译，输入为源语言的token序列，输出为目标语言的token序列。整个模型有两个简单循环神经网络$f_1(\cdot)$和$f_2(\cdot)$以及一个分类器$g(\cdot)$，模型的运算过程如下
  $$h_t = f_1(h_{t - 1}, x_t)\tag{8}$$
  
  $$h_{T + t} = f_2(h_{T + t - 1}, \hat{y}_{t-1})\tag{9}$$
  
  $$\hat{y}_t = g(h_{T + t})\tag{10}$$
  $\hat{y}_0$不是计算出来的，而是输入序列的结束符\<EOS>

## 参数学习
RNN的参数通过梯度下降法进行学习，梯度计算方式主要有两种

### 随时间反向传播算法(BPTT)

给定一个样本$(x, y)$，其中$x_{1: T} = (x_1, x_2,\ldots, x_T)$是长度为T的输入序列，$y_{1: T} = (x_1, x_2,\ldots, x_T)$是长度为T的标签序列，即在每个时刻$t$，都有一个监督信息$y_t$，定义时刻$t$的损失函数为
$$
\mathcal{L}_t = \mathcal{L}(y_t, g(h_t))\tag{11}
$$

其中$g(h_t)$为第$t$时刻的输出，$L$为可微分的损失函数。那么整个序列的损失为

$$
\mathcal{L} = \displaystyle\sum_{t = 1} \mathcal{L}_t\tag{12}
$$

整个序列的损失函数$\mathcal{L}$关于参数$U$的梯度为
$$
\frac{\partial\mathcal{L}}{\partial U} =\displaystyle\sum_{t = 1}^T \frac{\partial \mathcal{L}_t}{\partial U}\tag{13}
$$

由公式(2)可以看出，参数$U$和神经元在每个时刻$k(1\le k\le t)$的净输入有关，因此，第$t$时刻的损失函数关于参数$U$的梯度为：

$$
\frac{\partial\mathcal{L}}{\partial U} = \displaystyle\sum_{k = 1}^t\frac{\partial^+z_k}{\partial U}\frac{\partial\mathcal{L}_t}{\partial z_k}\tag{14}
$$

其中$\frac{\partial^+z_k}{\partial U}$表示“直接”偏导数，即公式(2)中保持$h_{k - 1}$不变.

$\frac{\partial^+z_k}{\partial U}$进一步可以写成
$$
\frac{\partial^+z_k}{\partial U} = h_{k - 1}^T\tag{15}
$$

定义误差项$\delta_{t, k} = \frac{\partial \mathcal{L}_t}{\partial z_k}$，那么展开误差项后有
$$
\begin{aligned}
\delta_{t, k} =& \frac{\partial \mathcal{L}_t}{\partial z_k}\\
=& \frac{\partial h_k}{\partial z_k}\frac{\partial z_{k + 1}}{\partial h_k}\frac{\partial \mathcal{L}}{\partial z_{k + 1}}\\
=& \text{diag}\left(f'(z_k)\right)U^T\delta_{t, k + 1}
\end{aligned}\tag{16}
$$

那么，综合公式(13)、(14)、(15)、(16)可以得到
$$
\frac{\partial \mathcal{L}}{\partial U} = \displaystyle\sum_{t = 1}^T\sum_{k = 1}^t\delta_{t, k}h^T_{k - 1}\tag{17}
$$

同理可得，$\mathcal{L}$关于权重$W$和偏置$b$的梯度为
$$
\frac{\partial \mathcal{L}}{\partial W} = \displaystyle\sum_{t = 1}^T\sum_{k = 1}^t\delta_{t, k}x^T_{k - 1}\tag{18}
$$

$$
\frac{\partial \mathcal{L}}{\partial b} = \displaystyle\sum_{t = 1}^T\sum_{k = 1}^t\delta_{t, k}\tag{19}
$$

### 实时循环学习算法(RTRL)

与反向传播的BPTT不同，RTRL是通过前向传播来更新梯度。

假设RNN中第$t + 1$时刻的状态$h_{t + 1}$为
$$
h_{t  + 1} = f(Uh_t + Wx_{t + 1} + b)\tag{20}
$$

那么，它关于参数$U$的偏导数为
$$
\begin{aligned}
\frac{\partial h_{t + 1}}{\partial U}=&
\left(\frac{\partial^+z_{t + 1} }{\partial U} + \frac{\partial h_t}{\partial U}U^T \right)\frac{\partial h_{t + 1}}{\partial z_{t + 1}}\\
=&\left(h_t\otimes I_U + \frac{\partial h_t}{\partial U}U^T\right)\text{diag}(f'(z_{t + 1}))
\end{aligned}\tag{21}
$$

请注意，公式(21)中的$\otimes$不是卷积操作，而是kronecker积操作, 由于上述公式涉及到向量对矩阵求导，请参考[矩阵求导](../math/derivative.ipynb)

同理可以求得$h_{t + 1}$关于权重$W$和偏置$b$的梯度

$$
\begin{aligned}
\frac{\partial h_{t + 1}}{\partial W}=&
\frac{\partial^+z_{t + 1} }{\partial W}\frac{\partial h_{t + 1}}{\partial z_{t + 1}}\\
=&\left(x_{t+ 1}\otimes I_U \right)\text{diag}(f'(z_{t + 1}))
\end{aligned}\tag{22}
$$

$$
\frac{\partial h_{t + 1}}{\partial b}= f'(z_{t + 1})\tag{23}
$$

## RNN存在的问题

循环神经网络在学习过程中的主要问题是由于梯度消失或爆炸问题,很难建模长时间间隔的状态之间的依赖关系.

在BPTT算法中，将公式(16)展开得到
$$
\delta_{t, k} = \displaystyle\prod_{\tau = k}^{t - 1}\left(\text{diag}(f'(z_{\tau}))U^T \right)\delta_{t, t}\tag{24}
$$

定义$\gamma \overset{\sim}{=} \|\text{diag}(f'(z_{\tau}))U^T \|$， 那么
$$
\delta_{t, k} \overset{\sim}{=}\gamma^{t - k}\delta_{t, t}\tag{25}
$$

  若$\gamma > 1$, 当$t - k \to \infty$时， $\gamma^{t - k} \to \infty$，所以梯也会变得很大。会造成系统不稳，称为梯度爆炸问题
  相反，若$\gamma < 1$, 当$t - k \to 0$时，梯度也会变得非常小，会出现梯度消失的问题。这里的梯度是指$\frac{\partial \mathcal{L}_t}{\partial h_k}$,如果$t - k$间隔比较大，而循环神经元通常采用Sigmoid或者Tanh作为非线性激活函数，其导数值都小于1，所欲经常会出现梯度消失的问题。

1. 梯度爆炸，一般而言，循环网络的梯度爆炸问题较容易解决，通过权重衰减或者梯度截断来避免。
  权重衰减是通过正则化的方法来限制参数的取值范围，梯度阶段是另一种有效的启发式方法，当梯度的规模大于一定的阈值，就将它阶段为一个较小的数。
2. 梯度消失是循环网络的主要问题, 除了使用一些技巧以外，更有效的方式就是改变模型，比如令$U = I$，同时令$ \frac{\partial h_t}{\partial h_{t - 1}}  = I$为单位矩阵,即
  $$ h_t = h_{t - 1} + g(x_t;\theta)\tag{26}$$
  公式(26)虽然能够避免梯度消失的问题，但是相邻时刻状态之前是线性关系降低了模型的表示能能力，所以，通常用下列公式
  $$ h_t = h_{t - 1} + g(x_t, h_{t - 1};\theta)\tag{27}$$

更好方案是在公式(27)的基础上引入门控机制来控制信息的积累速度，本笔记介绍了其中的[LSTM](lstm.ipynb)以及[GRU](gru.ipynb)